## This is Dimas Notebook

# COVID Detection from Xray Images

These code mainly adapted from https://www.pyimagesearch.com/2020/03/16/detecting-covid-19-in-x-ray-images-with-keras-tensorflow-and-deep-learning/

### Import the necessary packages

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os

### Construct the argument parser and parse the arguments

In [0]:
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True,
	help="path to input dataset")
ap.add_argument("-p", "--plot", type=str, default="plot.png",
	help="path to output loss/accuracy plot")
ap.add_argument("-m", "--model", type=str, default="covid19.model",
	help="path to output loss/accuracy plot")
args = vars(ap.parse_args())

### Initialize the initial learning rate, number of epochs to train for, and batch size

In [0]:
INIT_LR = 1e-3
EPOCHS = 25
BS = 8

### Grab the list of images in our dataset directory, then initialize the list of data (i.e., images) and class images

In [0]:
print("[INFO] loading images...")
imagePaths = list(paths.list_images(args["dataset"]))
data = []
labels = []

### Loop over the image paths

In [0]:
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# load the image, swap color channels, and resize it to be a fixed
	# 224x224 pixels while ignoring aspect ratio
	image = cv2.imread(imagePath)
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)

### Convert the data and labels to NumPy arrays while scaling the pixel intensities to the range [0, 255]

In [0]:
data = np.array(data) / 255.0
labels = np.array(labels)

### Perform one-hot encoding on the labels

In [0]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

### Partition the data into training and testing splits using 80% of the data for training and the remaining 20% for testing

In [0]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)

### Initialize the training data augmentation object

In [0]:
trainAug = ImageDataGenerator(
	rotation_range=15,
	fill_mode="nearest")

### Load the VGG16 network, ensuring the head FC layer sets are left off

In [0]:
baseModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

### Construct the head of the model that will be placed on top of the the base model

In [0]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

### Place the head FC model on top of the base model (this will become the actual model we will train)

In [0]:
model = Model(inputs=baseModel.input, outputs=headModel)

### Loop over all layers in the base model and freeze them so they will *not* be updated during the first training process

In [0]:
for layer in baseModel.layers:
	layer.trainable = False

### Compile our model

In [0]:
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

### Train the head of the network

In [0]:
print("[INFO] training head...")
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

### Make predictions on the testing set

In [0]:
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

### For each image in the testing set we need to find the index of the label with corresponding largest predicted probability

In [0]:
predIdxs = np.argmax(predIdxs, axis=1)

### Show a nicely formatted classification report

In [0]:
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

### Compute the confusion matrix and and use it to derive the raw accuracy, sensitivity, and specificity

In [0]:
cm = confusion_matrix(testY.argmax(axis=1), predIdxs)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])

### Show the confusion matrix, accuracy, sensitivity, and specificity

In [0]:
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))

### Plot the training loss and accuracy

In [0]:
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

### Serialize the model to disk

In [0]:
print("[INFO] saving COVID-19 detector model...")
model.save(args["model"], save_format="h5")